In [1]:
import pandas as pd
import json
from PIL import Image
import os

from sklearn.model_selection import train_test_split

In [2]:
# inputs

VAL_SIZE = 0.33
random_state = 42

PATH_IMG = "/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/images"
PATH_MASK = "/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks"

n_images_max = None # limit the number of images to process when debugging

ksize = (5, 5)
dst = 220
thresh = 255
tol = 0.2
threshold_area=1

In [3]:
CATEGORY_MAP = {'crack': '(255, 255, 255)'}

In [4]:
# Source code:
# https://github.com/chrise96/image-to-coco-json-converter/blob/master/src/create_annotations.py

from PIL import Image
import numpy as np
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import os
import json
import pandas as pd
import cv2


def process_image(path, ksize=(3, 3), dst=220, thresh=255):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, ksize, 0)
    thresh = cv2.threshold(blur, dst, thresh, cv2.THRESH_BINARY)[1]

    return Image.fromarray(thresh).convert("RGB")


def create_sub_masks(mask_image, width, height):
    # Initialize a dictionary of sub-masks indexed by RGB colors

    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x, y))[:3]

            # Check to see if we have created a sub-mask...
            pixel_str = str(pixel)
            sub_mask = sub_masks.get(pixel_str)
            if sub_mask is None:
                # Create a sub-mask (one bit per pixel) and add to the dictionary
                # Note: we add 1 pixel of padding in each direction
                # because the contours module doesn"t handle cases
                # where pixels bleed to the edge of the image

                sub_masks[pixel_str] = Image.new("1", (width + 2, height + 2))

            # Set the pixel value to 1 (default is 0), accounting for padding
            sub_masks[pixel_str].putpixel((x + 1, y + 1), 1)

    return sub_masks


def create_sub_mask_annotation(sub_mask, tol, threshold_area):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(np.array(sub_mask), 0.5, positive_orientation="low")

    polygons = []
    segmentations = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)

        poly = poly.simplify(tol, preserve_topology=True)

        if (poly.is_empty) or poly.area < threshold_area:
            # Go to next iteration, dont save empty values in list
            continue
        #print('poly.area', poly.area)
        polygons.append(poly)

        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)

    return polygons, segmentations


def create_category_annotation(category_dict):
    category_list = []

    for key, value in category_dict.items():
        category = {
            "supercategory": key,
            "id": value,
            "name": key
        }
        category_list.append(category)

    return category_list


def create_image_annotation(file_name, width, height, image_id):
    images = {
        "file_name": file_name,
        "height": height,
        "width": width,
        "id": image_id
    }

    return images


def create_annotation_format(polygon, segmentation, image_id, category_id, annotation_id):
    min_x, min_y, max_x, max_y = polygon.bounds
    width = max_x - min_x
    height = max_y - min_y
    bbox = (min_x, min_y, width, height)
    area = polygon.area

    annotation = {
        "segmentation": segmentation,
        "area": area,
        "iscrowd": 0,
        "image_id": image_id,
        "bbox": bbox,
        "category_id": category_id,
        "id": annotation_id
    }

    return annotation


def get_coco_json_format():
    # Standard COCO format
    coco_format = {
        "info": {},
        "licenses": [],
        "images": [{}],
        "categories": [{}],
        "annotations": [{}]
    }

    return coco_format


def clean_polygons(polygons):
    for p1 in polygons:
        for p2 in polygons:
            if p1 != p2:
                inter_area = p1.intersection(p2).area
                union_area = p1.union(p2).area

                # print('inter_area', inter_area)
                # print('union_area', union_area)
                # if union_area>0:
                # print('ratio', inter_area/union_area)

                if union_area > 0 and inter_area / union_area > 0:
                    # print('ratio', inter_area/union_area)
                    if p1.area > p2.area:
                        polygons.remove(p2)
                    else:
                        polygons.remove(p1)
    return polygons


def images_annotations_info(
        paths_df, category_colors, multipolygon_ids,
        n_images_max=None,
        ksize=(5, 5), dst=220, thresh=255,
        tol=1, threshold_area=10):
    
    annotation_id = 0
    annotations = []
    images = []
    images_to_drop = []

    n_images = paths_df.shape[0] if n_images_max == None else n_images_max

    for image_id in range(n_images):
        line = paths_df.iloc[image_id]

        print('image {}/{}'.format(image_id + 1, n_images))

        # mask_image_open = Image.open(line.path_mask).convert("RGB")
        mask_image_open = process_image(line.path_mask, ksize, dst, thresh)
        print(line.path_mask)

        w, h = mask_image_open.size

        # "images" info
        image = create_image_annotation(line.path_image, w, h, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open, w, h)
        
        if len(set(category_colors.keys()).intersection(sub_masks.keys()))==0:
            images_to_drop.append(line.image)
            continue
        
        for color, sub_mask in sub_masks.items():

            if color in category_colors.keys():

                category_id = category_colors[color]                
                polygons, segmentations = create_sub_mask_annotation(sub_mask, tol, threshold_area)
                
                if len(polygons)==0:
                    images_to_drop.append(line.image)
                    continue

                if category_id in multipolygon_ids:
                    
                    polygons = clean_polygons(polygons)
                    multi_poly = MultiPolygon(polygons)

                    annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id,
                                                          annotation_id)
                    annotations.append(annotation)
                    annotation_id += 1

        image_id += 1

        if n_images_max is not None and n_images_max <= image_id:
            break
            
    paths_df = paths_df[~paths_df.image.isin(images_to_drop)].reset_index(drop=True)

    return paths_df, images, annotations, annotation_id

In [5]:
category_ids = {cat: i+1 for i,cat in enumerate(CATEGORY_MAP.keys())}
category_colors = {color: i+1 for i,color in enumerate(CATEGORY_MAP.values())}
multipolygon_ids = list(category_ids.values())

In [6]:
files_imgs = [os.path.join(PATH_IMG, p) for p in os.listdir(PATH_IMG)]
files_masks = [os.path.join(PATH_MASK, p) for p in os.listdir(PATH_MASK)]

paths_df = pd.DataFrame(files_imgs, columns = ['path_image'])
paths_df['image'] = paths_df.path_image.apply(lambda x: x.split('/')[-1])

paths_m_df = pd.DataFrame(files_masks, columns = ['path_mask'])
paths_m_df['image'] = paths_m_df.path_mask.apply(lambda x: x.split('/')[-1])

paths_df = paths_df.merge(paths_m_df, on='image', how='inner')

In [7]:
paths_df.head()

,path_image,image,path_mask
0,/kaggle/input/crack-segmentation-dataset/crack...,DeepCrack_IMG_6469-7.jpg,/kaggle/input/crack-segmentation-dataset/crack...
1,/kaggle/input/crack-segmentation-dataset/crack...,Rissbilder_for_Florian_9S6A2828_80_1698_3421_3...,/kaggle/input/crack-segmentation-dataset/crack...
2,/kaggle/input/crack-segmentation-dataset/crack...,Rissbilder_for_Florian_9S6A2880_131_2019_3358_...,/kaggle/input/crack-segmentation-dataset/crack...
3,/kaggle/input/crack-segmentation-dataset/crack...,CRACK500_20160222_164851_641_1.jpg,/kaggle/input/crack-segmentation-dataset/crack...
4,/kaggle/input/crack-segmentation-dataset/crack...,CRACK500_20160316_144203_1921_721.jpg,/kaggle/input/crack-segmentation-dataset/crack...


In [8]:
paths_train_df, paths_val_df = train_test_split(
         paths_df, test_size=VAL_SIZE, random_state=random_state)

In [9]:
coco_format = get_coco_json_format()
coco_format["categories"] = create_category_annotation(category_ids)
paths_train_df, coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(
    paths_train_df, category_colors, multipolygon_ids, n_images_max, ksize, dst, thresh, tol, threshold_area)

with open('annotations_train.json', 'w') as f:
    json.dump(coco_format, f)
    
coco_format = get_coco_json_format()
coco_format["categories"] = create_category_annotation(category_ids)
paths_val_df, coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(
    paths_val_df, category_colors, multipolygon_ids,n_images_max, ksize, dst, thresh, tol, threshold_area)

with open('annotations_val.json', 'w') as f:
    json.dump(coco_format, f)

image 1/7569
/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks/CRACK500_20160310_114459_1281_1.jpg
image 2/7569
/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks/Volker_DSC01645_184_50_2098_1679.jpg
image 3/7569
/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks/Volker_DSC01702_148_426_1258_1569.jpg
image 4/7569
/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks/CRACK500_20160222_163940_641_1081.jpg
image 5/7569
/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks/forest_030.jpg
image 6/7569
/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks/Rissbilder_for_Florian_9S6A3102_15_2298_3780_2854.jpg
image 7/7569
/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks/Volker_DSC01610_123_687_1549_1614.jpg
image 8/7569
/kaggle/input/crack-segmentation-dataset/crack_segmentation_dataset/masks/CRACK500_20160405_171410_1921_721.jpg
image 9/75

In [10]:
paths_train_df['usage'] = 'train'
paths_val_df['usage'] = 'valid'

paths_df = pd.concat([paths_train_df, paths_val_df], axis=0)
paths_df.to_csv('paths.csv', index=False)

### testing code

paths_df[paths_df.image=='noncrack_noncrack_concrete_wall_24_32.jpg.jpg']

image_id = 8089 #2262
line = paths_df.loc[image_id]


display(Image.open(line.path_image).convert('RGB'))
#mask_image_open = Image.open(line.path_mask).convert('RGB')
mask_image_open = process_image(line.path_mask, ksize = (5,5), dst = 220, thresh=255)
display(mask_image_open)
w, h = mask_image_open.size
sub_masks = create_sub_masks(mask_image_open, w, h)
sub_mask = sub_masks['(255, 255, 255)']

polygons, segmentations = create_sub_mask_annotation(sub_mask,  tol=0.2, threshold_area=1)
polygons = clean_polygons(polygons)
multi_poly = MultiPolygon(polygons)
multi_poly

In [11]:
589

589